In [10]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap

In [11]:
# Load the biodiversity hotspots shapefile
hotspots = gpd.read_file('data/hotspots_2016_1/hotspots_2016_1.shp')

In [12]:
# Load the biodiversity hotspots shapefile for Amazon Rainforest
# Paths to all shapefiles
shapefile_paths = [
    'data/ANps_2020/ANP_BosqueProtector.shp',
    'data/ANps_2020/ANP_departamental.shp',
    'data/ANps_2020/ANP_nacional_puntual.shp',
    'data/ANps_2020/ANP_nacional.shp',
    'data/ANps_2020/ANP_ReservaFlorestal.shp'
]
# Load each shapefile into a GeoDataFrame
gdfs = [gpd.read_file(shp) for shp in shapefile_paths]



In [13]:
# Ensure all GeoDataFrames have the same CRS (Coordinate Reference System)
crs = gdfs[0].crs  # Use the CRS from the first shapefile
for gdf in gdfs:
    if gdf.crs != crs:
        gdf.to_crs(crs, inplace=True)

# Merge all GeoDataFrames into one using pd.concat()
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
merged_gdf.head(5)

,pais,categoria,nombre,norma,fecha_crea,fecha_atua,uso,status,ambito_adm,categoria_,area_sig_h,fuente,fecha_at_1,leyenda,institucio,link,amzbiog,limraisg,geometry,objectid
0,Ecuador,Bosque Protector,MORAVIA,R.O. 172 - 14 oct 1997 / R.M. 29 - 11 jul 1997,1997,0.0,directo/indirecto,creado,nacional,0.0,605.448408,"MAE, 2018",0,uso directo/indirecto,ECOCIENCIA,None,s,s,"POLYGON ((-78.03713 -1.46722, -78.03713 -1.485...",NaN
1,Ecuador,Bosque Protector,BIOGEOCULTURAL TUNA KARAMMA,R.O. 187 - 9 oct 2003 / R.M. 111 - 16 sep 2003,2003,0.0,directo/indirecto,creado,nacional,0.0,400.498885,"MAE, 2018",0,uso directo/indirecto,ECOCIENCIA,None,s,s,"POLYGON ((-78.09978 -1.86954, -78.09981 -1.869...",NaN
2,Ecuador,Bosque Protector,COLAMBO - YACURI,R.O. 519 - 21 feb 2002 / R.M. 17 - 30 ene 2002,2002,0.0,directo/indirecto,creado,nacional,0.0,33349.802019,"MAE, 2018",0,uso directo/indirecto,ECOCIENCIA,None,s,s,"MULTIPOLYGON (((-79.36623 -4.79703, -79.36579 ...",NaN
3,Ecuador,Bosque Protector,PANACOCHA,R.O. 472 - 29 jun 1994,1994,0.0,directo/indirecto,creado,nacional,0.0,59367.399217,"MAE, 2018",0,uso directo/indirecto,ECOCIENCIA,None,s,s,"POLYGON ((-76.16882 -0.28858, -76.16882 -0.288...",NaN
4,Ecuador,Bosque Protector,CERRO CASIGANA,R.O. 8 - 22 ago 1984 / R.M. 364 - 5 jul 1984,1984,0.0,directo/indirecto,creado,nacional,0.0,175.908290,"MAE, 2018",0,uso directo/indirecto,ECOCIENCIA,None,n,s,"POLYGON ((-78.64543 -1.25676, -78.64534 -1.256...",NaN


In [14]:
# Load the protected areas CSV data
protected_areas = pd.read_csv('data/WDPA_WDOECM_Sep2024_Public_EU_csv.csv')

# Check if the protected areas CSV contains latitude and longitude for spatial plotting
print(protected_areas.columns)


Index(['TYPE', 'WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME', 'ORIG_NAME', 'DESIG',
       'DESIG_ENG', 'DESIG_TYPE', 'IUCN_CAT', 'INT_CRIT', 'MARINE',
       'REP_M_AREA', 'GIS_M_AREA', 'REP_AREA', 'GIS_AREA', 'NO_TAKE',
       'NO_TK_AREA', 'STATUS', 'STATUS_YR', 'GOV_TYPE', 'OWN_TYPE',
       'MANG_AUTH', 'MANG_PLAN', 'VERIF', 'METADATAID', 'SUB_LOC',
       'PARENT_ISO3', 'ISO3', 'SUPP_INFO', 'CONS_OBJ'],
      dtype='object')


/var/folders/q2/v8zk8rk54691thc1c5rrpbjm0000gn/T/ipykernel_84036/3847323171.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  protected_areas = pd.read_csv('data/WDPA_WDOECM_Sep2024_Public_EU_csv.csv')


In [15]:
# Assuming the CSV has 'LATITUDE' and 'LONGITUDE' columns for protected areas
if 'LATITUDE' in protected_areas.columns and 'LONGITUDE' in protected_areas.columns:
    # Convert the protected areas DataFrame to a GeoDataFrame
    protected_areas_gdf = gpd.GeoDataFrame(
        protected_areas, 
        geometry=gpd.points_from_xy(protected_areas['LONGITUDE'], protected_areas['LATITUDE']),
        crs="EPSG:4326"  # Assuming WGS84 for lat/lon
    )
else:
    print("Protected areas CSV does not contain geographic coordinates.")
    protected_areas_gdf = None

# Create a Folium map centered on Europe
m = folium.Map(location=[54.5260, 15.2551], zoom_start=4)

# Convert the centroids of the biodiversity hotspots to a list of [lat, lon] for heatmap visualization
hotspots_coords = [[point.y, point.x] for point in hotspots.geometry.centroid]

# Add the biodiversity hotspots heatmap to the map
HeatMap(hotspots_coords, radius=10).add_to(m)

# If protected areas contain geographic data, overlay them on the map
if protected_areas_gdf is not None:
    # Add protected areas to the map
    folium.GeoJson(
        protected_areas_gdf,
        name="Protected Areas",
        style_function=lambda x: {'color': 'green', 'weight': 1.5, 'fillOpacity': 0.5}
    ).add_to(m)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('images/biodiversity_hotspots_and_protected_areas_europe.html')

# Display the map (if you're in a Jupyter environment)
m

Protected areas CSV does not contain geographic coordinates.


/var/folders/q2/v8zk8rk54691thc1c5rrpbjm0000gn/T/ipykernel_84036/1838675571.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hotspots_coords = [[point.y, point.x] for point in hotspots.geometry.centroid]


In [16]:
# Check the Coordinate Reference System (CRS) to ensure both layers match
if hotspots.crs != merged_gdf.crs:
    merged_gdf = merged_gdf.to_crs(hotspots.crs)

In [17]:
# Filter the biodiversity hotspots to focus on the Amazon region if needed
amazon_hotspots = hotspots.cx[-75:-45, -20:5]  # Example for Amazon area bounding box
amazon_hotspots

,NAME,Type,geometry
0,Atlantic Forest,hotspot area,"MULTIPOLYGON (((-48.73786 -26.28284, -48.73545..."
7,Cerrado,hotspot area,"MULTIPOLYGON (((-61.00920 -15.05946, -61.01486..."
44,Tropical Andes,hotspot area,"MULTIPOLYGON (((-66.76953 10.61377, -66.76758 ..."


In [18]:
# Create a Folium map centered around the Amazon region
m = folium.Map(location=[-3.4653, -62.2159], zoom_start=5)  # Coordinates in Amazon Rainforest

# Convert the centroids of the biodiversity hotspots to a list of [lat, lon] for heatmap visualization
hotspots_coords = [[point.y, point.x] for point in amazon_hotspots.geometry.centroid]

# Add the biodiversity hotspots heatmap to the map
HeatMap(hotspots_coords, radius=10).add_to(m)

# Add protected areas as polygons
folium.GeoJson(
    merged_gdf,
    name="Amazon Protected Areas",
    style_function=lambda x: {'color': 'green', 'weight': 1.5, 'fillOpacity': 0.4}
).add_to(m)

# Add layer control to toggle between layers
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('images/amazon_biodiversity_hotspots_and_protected_areas.html')

# Display the map (optional, works in Jupyter Notebooks)
m

/var/folders/q2/v8zk8rk54691thc1c5rrpbjm0000gn/T/ipykernel_84036/3755665236.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hotspots_coords = [[point.y, point.x] for point in amazon_hotspots.geometry.centroid]
